In [7]:
using PETLION
using CSV
using DataFrames
using Plots
using Logging
using Dates

In [8]:
@time p = petlion(
    LCO;
    aging =:SEI,
    N_p = 10, # di=scretizations in the cathode
    N_s = 10, # discretizations in the separator
    N_n = 10, # discretizations in the anode
    N_r_p = 10, # discretizations in the solid cathode particles
    N_r_n = 10 , # discretizations in the solid anode particles
    temperature = false, # temperature enabled or disabled
    jacobian = :AD, # :AD (automatic-differenation) for convenience or :symbolic for speed

)

 97.406683 seconds (336.96 M allocations: 21.676 GiB, 2.66% gc time, 10.17% compilation time)


PETLION model{jacobian_AD}:
  Cathode: LCO, rxn_BV, & OCV_LCO
  Anode:   LiC6, rxn_BV, & OCV_LiC6
  System:  D_s_eff, rxn_rate, D_eff_linear, K_eff, & thermodynamic_factor_linear
  --------
  Temperature:     false
  Solid diffusion: Fickian, finite_difference
  Aging:           SEI
  Voltage bounds:  [2.5 V, 4.3 V]
  SOC bounds:      [0.0, 1.0]
  --------
  N.p:   10
  N.r_p: 10
  N.s:   10
  N.n:   10
  N.r_n: 10

In [1]:
pwd()

"/Users/wuzhengrun/Desktop/jupyter_notebooks/0924sampling"

In [4]:
Dates.format(now(), "HH:MM")  

"15:41"

In [10]:
mkdir("/Users/wuzhengrun/Desktop/jupyter_notebooks/0924samplingdata")
for m in 0:9  
    println(Dates.format(now(), "HH:MM"))
    mkdir("/Users/wuzhengrun/Desktop/jupyter_notebooks/0924samplingdata/soc_start_$m") 
    for n in m+1:10
        mkdir("/Users/wuzhengrun/Desktop/jupyter_notebooks/0924samplingdata/soc_start_$m/soc_end_$n") #  rm -r cpcv0923
        for j in 0:20
            empty!(p);PETLION.options[:FACTORIZATION_METHOD] = :LU
            tempP = 350-5*j
            # cp充电
            V_max1 = 4.1
            set_SOC_start = m/10
            set_SOC_end = n/10
            try        
                sol = simulate(p,  P=tempP, SOC=set_SOC_start,SOC_max = set_SOC_end, V_max=V_max1, outputs=:all)
                simulate!(sol, p, V=:hold ,I_min=1/20,SOC_max = set_SOC_end)
                start_0=1
                set_SOH = sol.SOH[start_0] #  初始SOH
                delta_SOH = sol.SOH[end] - sol.SOH[start_0]
                SOC_end = sol.SOC[end]
                df = DataFrame(set_V=V_max1, set_P=tempP,m=m,n=n, SOC_start=set_SOC_start,SOC_end=SOC_end, set_SOH=set_SOH, SOH_end = sol.SOH[end], delta_SOH=delta_SOH)
                temp_cycle = 0
                for i in 1:2550
                    try
                    # cp放电
                        simulate!(sol, p, P=-tempP, V_min=2.5, SOC_min = set_SOC_start) 
                        simulate!(sol, p, V=:hold, I_max=-1/20, SOC_min = set_SOC_start)
                        start_0=length(sol.t)+1
                    # cp充电 SOC=set_SOC_start SOC不应该限制start，而是由上一步的放电得出
                        simulate!(sol, p, P=tempP, V_max=V_max1, SOC_max = set_SOC_end, outputs=:all)
                        simulate!(sol, p, V=:hold,SOC_max = set_SOC_end, I_min=1/20)    
                        set_SOH = sol.SOH[start_0] #  初始SOH
                        delta_SOH = sol.SOH[end] - sol.SOH[start_0]
                        push!(df,[V_max1, tempP,m,n,sol.SOC[start_0], sol.SOC[end], set_SOH, sol.SOH[end], delta_SOH])
                        start_0=length(sol.t)+1  

                        if sol.SOH[end]<0.8
                            println("Cycle Finish!\ttempP:  ",tempP,"\tSOC_start",sol.SOC[end],"\tSOC[end]",sol.SOC[end], "\ttemp_cycle:   ", i,"\tm n",m,n)
                            break 
                        end
                    catch ex
                        print(ex)
                        println("i:   ", i,"   ", V_max1, tempP)
                        break
                    end  
                end#SOH C的积分 for i的end
#                 print(df)
                CSV.write("/Users/wuzhengrun/Desktop/jupyter_notebooks/0924samplingdata/soc_start_$m/soc_end_$n/P_$tempP.csv",df) 

            catch ex
                print(ex)
                println("j: ",j,"   ", V_max1,"tempP is  ", tempP)
            continue
            end

        end # end for j 
    end # end for n
end # end for m

15:46
ErrorException("Model failed to converge at t = 63.15881519544544")i:   20   4.1350
ErrorException("Model failed to converge at t = 77.23785386749563")i:   1520   4.1330
ErrorException("Model failed to converge at t = 76.56367227891137")i:   895   4.1310
ErrorException("Model failed to converge at t = 84.88197261222268")i:   304   4.1290
ErrorException("Model failed to converge at t = 89.3363046084323")i:   758   4.1285
ErrorException("Model failed to converge at t = 95.6780345035168")i:   715   4.1275
ErrorException("Model failed to converge at t = 88.45240281702696")i:   138   4.1270
ErrorException("Model failed to converge at t = 128.63214606605538")i:   262   4.1350
ErrorException("Model failed to converge at t = 144.5154383989799")i:   23   4.1340
ErrorException("Model failed to converge at t = 145.73266359476813")i:   251   4.1330
ErrorException("Model failed to converge at t = 145.80089928200795")i:   248   4.1325
ErrorException("Model failed to converge at t = 161.5994135

ErrorException("Model failed to converge at t = 457.465178773066")i:   87   4.1305
ErrorException("Model failed to converge at t = 467.54185401935683")i:   106   4.1300
Cycle Finish!	tempP:  295	SOC_start0.6006852969436469	SOC[end]0.6006852969436469	temp_cycle:   219	m n06
Cycle Finish!	tempP:  290	SOC_start0.6006860827398203	SOC[end]0.6006860827398203	temp_cycle:   223	m n06
Cycle Finish!	tempP:  285	SOC_start0.6006816425810035	SOC[end]0.6006816425810035	temp_cycle:   230	m n06
Cycle Finish!	tempP:  280	SOC_start0.6006604523867034	SOC[end]0.6006604523867034	temp_cycle:   237	m n06
ErrorException("Model failed to converge at t = 528.322695586182")i:   8   4.1275
Cycle Finish!	tempP:  270	SOC_start0.6005084564623729	SOC[end]0.6005084564623729	temp_cycle:   253	m n06
ErrorException("Model failed to converge at t = 601.4413162848255")i:   26   4.1265
ErrorException("Model failed to converge at t = 575.2564812732478")i:   61   4.1260
ErrorException("Model failed to converge at t = 615.8887

Cycle Finish!	tempP:  315	SOC_start0.9994238097031006	SOC[end]0.9994238097031006	temp_cycle:   99	m n010
Cycle Finish!	tempP:  310	SOC_start0.9994122860063314	SOC[end]0.9994122860063314	temp_cycle:   100	m n010
ErrorException("Model failed to converge at t = 282.0082353193395")i:   61   4.1305
Cycle Finish!	tempP:  300	SOC_start0.9992797289078506	SOC[end]0.9992797289078506	temp_cycle:   104	m n010
Cycle Finish!	tempP:  295	SOC_start0.999404627845861	SOC[end]0.999404627845861	temp_cycle:   105	m n010
Cycle Finish!	tempP:  290	SOC_start1.000026307203505	SOC[end]1.000026307203505	temp_cycle:   107	m n010
Cycle Finish!	tempP:  285	SOC_start0.9997411403711324	SOC[end]0.9997411403711324	temp_cycle:   109	m n010
ErrorException("Model failed to converge at t = 25.05016636640045")i:   57   4.1280
Cycle Finish!	tempP:  275	SOC_start0.9996869488032918	SOC[end]0.9996869488032918	temp_cycle:   114	m n010
Cycle Finish!	tempP:  270	SOC_start0.999526270676079	SOC[end]0.999526270676079	temp_cycle:   11

ErrorException("Model failed to converge at t = 404.8129349199922")i:   103   4.1335
Cycle Finish!	tempP:  330	SOC_start0.6007577426864343	SOC[end]0.6007577426864343	temp_cycle:   182	m n16
ErrorException("Model failed to converge at t = 410.9075804686811")i:   72   4.1325
Cycle Finish!	tempP:  320	SOC_start0.6007567961400948	SOC[end]0.6007567961400948	temp_cycle:   194	m n16
Cycle Finish!	tempP:  315	SOC_start0.6006447893793436	SOC[end]0.6006447893793436	temp_cycle:   199	m n16
Cycle Finish!	tempP:  310	SOC_start0.6006331279325117	SOC[end]0.6006331279325117	temp_cycle:   202	m n16
Cycle Finish!	tempP:  305	SOC_start0.6007138916415236	SOC[end]0.6007138916415236	temp_cycle:   208	m n16
Cycle Finish!	tempP:  300	SOC_start0.6006828632088219	SOC[end]0.6006828632088219	temp_cycle:   213	m n16
Cycle Finish!	tempP:  295	SOC_start0.6006847738889647	SOC[end]0.6006847738889647	temp_cycle:   221	m n16
Cycle Finish!	tempP:  290	SOC_start0.6006855742331252	SOC[end]0.6006855742331252	temp_cycle:   2

Cycle Finish!	tempP:  340	SOC_start0.9990463789770424	SOC[end]0.9990463789770424	temp_cycle:   93	m n110
Cycle Finish!	tempP:  335	SOC_start0.9994905649676988	SOC[end]0.9994905649676988	temp_cycle:   94	m n110
Cycle Finish!	tempP:  330	SOC_start0.9994751148778206	SOC[end]0.9994751148778206	temp_cycle:   96	m n110
ErrorException("Model failed to converge at t = 9.454936260907742")i:   95   4.1325
ErrorException("Model failed to converge at t = 59.56754131717928")i:   34   4.1320
ErrorException("Model failed to converge at t = 61.56698916691694")i:   85   4.1315
Cycle Finish!	tempP:  310	SOC_start1.000162820608362	SOC[end]1.000162820608362	temp_cycle:   103	m n110
Cycle Finish!	tempP:  305	SOC_start0.9994785322390098	SOC[end]0.9994785322390098	temp_cycle:   104	m n110
Cycle Finish!	tempP:  300	SOC_start0.9996237028518399	SOC[end]0.9996237028518399	temp_cycle:   106	m n110
Cycle Finish!	tempP:  295	SOC_start0.9996835874249215	SOC[end]0.9996835874249215	temp_cycle:   108	m n110
Cycle Finis

...

Cycle Finish!	tempP:  345	SOC_start0.799832823505536	SOC[end]0.799832823505536	temp_cycle:   146	m n48
Cycle Finish!	tempP:  340	SOC_start0.7997672714362581	SOC[end]0.7997672714362581	temp_cycle:   149	m n48
Cycle Finish!	tempP:  335	SOC_start0.8000412999117975	SOC[end]0.8000412999117975	temp_cycle:   152	m n48
Cycle Finish!	tempP:  330	SOC_start0.8000281626527219	SOC[end]0.8000281626527219	temp_cycle:   155	m n48
Cycle Finish!	tempP:  325	SOC_start0.7997834686059082	SOC[end]0.7997834686059082	temp_cycle:   157	m n48
Cycle Finish!	tempP:  320	SOC_start0.8000353801830018	SOC[end]0.8000353801830018	temp_cycle:   161	m n48
Cycle Finish!	tempP:  315	SOC_start0.799823257161196	SOC[end]0.799823257161196	temp_cycle:   162	m n48
Cycle Finish!	tempP:  310	SOC_start0.799789805150574	SOC[end]0.799789805150574	temp_cycle:   166	m n48
Cycle Finish!	tempP:  305	SOC_start0.7997687987076674	SOC[end]0.7997687987076674	temp_cycle:   169	m n48
Cycle Finish!	tempP:  300	SOC_start0.7997668050978138	SOC[end

Cycle Finish!	tempP:  265	SOC_start0.6006198417947906	SOC[end]0.6006198417947906	temp_cycle:   848	m n56
Cycle Finish!	tempP:  260	SOC_start0.6006131115149435	SOC[end]0.6006131115149435	temp_cycle:   871	m n56
Cycle Finish!	tempP:  255	SOC_start0.6006022537441152	SOC[end]0.6006022537441152	temp_cycle:   898	m n56
Cycle Finish!	tempP:  250	SOC_start0.6005938866522181	SOC[end]0.6005938866522181	temp_cycle:   925	m n56
Cycle Finish!	tempP:  350	SOC_start0.7005406835695612	SOC[end]0.7005406835695612	temp_cycle:   263	m n57
Cycle Finish!	tempP:  345	SOC_start0.7008026904225556	SOC[end]0.7008026904225556	temp_cycle:   269	m n57
Cycle Finish!	tempP:  340	SOC_start0.7007696034813178	SOC[end]0.7007696034813178	temp_cycle:   274	m n57
Cycle Finish!	tempP:  335	SOC_start0.7007568618471851	SOC[end]0.7007568618471851	temp_cycle:   280	m n57
Cycle Finish!	tempP:  330	SOC_start0.7007564584764087	SOC[end]0.7007564584764087	temp_cycle:   286	m n57
Cycle Finish!	tempP:  325	SOC_start0.7007637967018444	S

Cycle Finish!	tempP:  295	SOC_start0.9997714292857764	SOC[end]0.9997714292857764	temp_cycle:   177	m n510
Cycle Finish!	tempP:  290	SOC_start0.9997447624586467	SOC[end]0.9997447624586467	temp_cycle:   181	m n510
Cycle Finish!	tempP:  285	SOC_start0.9996935812334503	SOC[end]0.9996935812334503	temp_cycle:   182	m n510
Cycle Finish!	tempP:  280	SOC_start0.9996552423615463	SOC[end]0.9996552423615463	temp_cycle:   186	m n510
Cycle Finish!	tempP:  275	SOC_start0.9996980183218867	SOC[end]0.9996980183218867	temp_cycle:   188	m n510
Cycle Finish!	tempP:  270	SOC_start0.9998398102881184	SOC[end]0.9998398102881184	temp_cycle:   192	m n510
Cycle Finish!	tempP:  265	SOC_start0.9996812226623726	SOC[end]0.9996812226623726	temp_cycle:   196	m n510
Cycle Finish!	tempP:  260	SOC_start0.9996852197432898	SOC[end]0.9996852197432898	temp_cycle:   200	m n510
Cycle Finish!	tempP:  255	SOC_start0.9998658537677787	SOC[end]0.9998658537677787	temp_cycle:   204	m n510
Cycle Finish!	tempP:  250	SOC_start1.000068334

Cycle Finish!	tempP:  325	SOC_start0.9998274496680449	SOC[end]0.9998274496680449	temp_cycle:   195	m n610
Cycle Finish!	tempP:  320	SOC_start0.9999541376421737	SOC[end]0.9999541376421737	temp_cycle:   201	m n610
Cycle Finish!	tempP:  315	SOC_start1.0000261267400776	SOC[end]1.0000261267400776	temp_cycle:   207	m n610
Cycle Finish!	tempP:  310	SOC_start0.9997732615271222	SOC[end]0.9997732615271222	temp_cycle:   214	m n610
Cycle Finish!	tempP:  305	SOC_start0.9996579557059355	SOC[end]0.9996579557059355	temp_cycle:   224	m n610
Cycle Finish!	tempP:  300	SOC_start0.9996527320150506	SOC[end]0.9996527320150506	temp_cycle:   229	m n610
Cycle Finish!	tempP:  295	SOC_start0.9996529577454536	SOC[end]0.9996529577454536	temp_cycle:   232	m n610
Cycle Finish!	tempP:  290	SOC_start0.9995669876415787	SOC[end]0.9995669876415787	temp_cycle:   235	m n610
Cycle Finish!	tempP:  285	SOC_start0.9995765794257818	SOC[end]0.9995765794257818	temp_cycle:   244	m n610
Cycle Finish!	tempP:  280	SOC_start0.999582541

Cycle Finish!	tempP:  250	SOC_start0.9997990055266712	SOC[end]0.9997990055266712	temp_cycle:   395	m n710
18:27
Cycle Finish!	tempP:  350	SOC_start0.9000181270714639	SOC[end]0.9000181270714639	temp_cycle:   898	m n89
Cycle Finish!	tempP:  345	SOC_start0.8999648030342464	SOC[end]0.8999648030342464	temp_cycle:   931	m n89
Cycle Finish!	tempP:  340	SOC_start0.8999474011316344	SOC[end]0.8999474011316344	temp_cycle:   948	m n89
Cycle Finish!	tempP:  335	SOC_start0.899995031073361	SOC[end]0.899995031073361	temp_cycle:   902	m n89
Cycle Finish!	tempP:  330	SOC_start0.9000226431871942	SOC[end]0.9000226431871942	temp_cycle:   905	m n89
Cycle Finish!	tempP:  325	SOC_start0.900022233155627	SOC[end]0.900022233155627	temp_cycle:   909	m n89
Cycle Finish!	tempP:  320	SOC_start0.9000229647995052	SOC[end]0.9000229647995052	temp_cycle:   934	m n89
Cycle Finish!	tempP:  315	SOC_start0.8999896006177397	SOC[end]0.8999896006177397	temp_cycle:   967	m n89
Cycle Finish!	tempP:  310	SOC_start0.899977407760147

In [ ]:
270